In [2]:
import numpy as np
from Env import _5G
from DDQN_AGENT import *
from collections import deque   
from tqdm import tqdm
import os
import pickle
N= 5    #CUE тоо
M= 14   #DTUE тоо
MODEL_NAME = 'TEST_NUM03'#сургасан үр дүнгээ хадгалах нэр(log/ дотор хадгалагдана)

EPISODES = 5000 #Хэдэн state дээр ажиллахаа энд тохируулна
NUM_OF_ITER = 100 #Нэг state дээр хэдэн удаа allocation хийхээ энд тохируулна


EPSILON_DECAY = 0.995   #Нэг episode бүрийн эцсийн Decay
MIN_EPSILON = 0.01      #Хамгийн бага epsilona
Smoothing_memory = 100  #Үр дүгн сүүлийн Smoothing_memory утгаар дундаж(минимум, максимум) утгыг харах



MODEL_NAME = 'DDQN_' + MODEL_NAME
env = _5G(M,N)
agent = DDQN_Agent(M, N, MODEL_NAME)
starting_episode = 1
min_total= np.inf
total_reward= deque(maxlen=Smoothing_memory)
total_memory= deque(maxlen=Smoothing_memory)    
epsilon = 1

file_path = 'saved_weights/' + MODEL_NAME + 'primary.index'
if os.path.exists(file_path):
    agent.model.load_weights('saved_weights/' + MODEL_NAME + 'primary')
    agent.target_model.load_weights('saved_weights/' + MODEL_NAME + 'target')
    with open('saved_weights/step_' + MODEL_NAME + '.txt', 'rb') as file:
        f = pickle.load(file)
        starting_episode = f[1]
        step = f[0]
        agent.replay_memory = deque(f[2], maxlen=REPLAY_MEMORY_SIZE)
    agent.tensorboard.step = int(step)
    with open('saved_weights/total_' + MODEL_NAME + '.txt', 'rb') as file:
        total_memory = pickle.load( file)
        total_memory = deque(total_memory, maxlen=Smoothing_memory)
    with open('saved_weights/reward_' + MODEL_NAME + '.txt', 'rb') as file:
        total_reward = pickle.load( file)
        total_reward = deque(total_reward, maxlen=Smoothing_memory)

try:
    for episode in tqdm(range(starting_episode, EPISODES + 1), ascii=False, unit='episodes'):


        current_state_ = env.reset()
        current_state = current_state_['DRUE'][np.argsort(current_state_['DRUE'][:, 0]), :]/40 + 0.5 
        current_state= np.concatenate((current_state, np.array(current_state_['DTUE'][np.argsort(current_state_['DTUE'][:, 0]), :])/1200 + 0.5))
        current_state= np.concatenate((current_state, np.array(current_state_['CUE'][np.argsort(current_state_['CUE'][:, 0]), :])/1200 + 0.5))

        for _ in range(NUM_OF_ITER):
            if np.random.random() > epsilon:
                Q = agent.get_qs(current_state)[0]
                action = np.array([np.argmax(i) for i in Q.reshape(M,N)])
            else:
                action = np.random.randint(0, N, size=(M,))
            
            #FORCED VALID ALLOCATION##############################
            #counter = 0
            # while not env.step(action)[1] and counter<50:
            #     action = np.random.randint(0, N, size=(M,))
            #     counter += 1
            #################################################

            done, tot, std = env.step(action)
            reward= 0
            #MEASURE REWARD
            if tot:
                if tot < min_total:
                    min_total = tot
                total_memory.append(tot)
                dr = tot - min_total if len(total_memory) > 1 else 0
                reward = dr if dr >= 0 else -std * 2
                total_reward.append(reward)
            else:
                reward = -10
                total_reward.append(reward)


            if total_memory:
                #MONITORING
                average_reward = np.mean(total_reward)
                min_reward = np.min(total_reward)
                max_reward = np.max(total_reward)
                min_total = np.min(total_memory)
                max_total = np.max(total_memory)
                avg_total = np.mean(total_memory)


                #EXPERIENCE BUFFER
                agent.update_replay_memory((current_state, action, reward, current_state, tot))
                agent.train()
                agent.tensorboard.update_stats(reward_avg=average_reward,
                                            reward_min=min_reward,
                                            reward_max=max_reward,
                                            total_avg=avg_total,
                                            total_max=max_total,
                                            total_min=min_total)

        print(epsilon)
        if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)
except KeyboardInterrupt:
    agent.model.save_weights('saved_weights/' + MODEL_NAME + 'primary')
    agent.target_model.save_weights('saved_weights/' + MODEL_NAME + 'target')
    with open('saved_weights/step_' + MODEL_NAME + '.txt', 'wb') as file:
        pickle.dump([agent.tensorboard.step, episode, agent.replay_memory], file)
    with open('saved_weights/total_' + MODEL_NAME + '.txt', 'wb') as file:
        pickle.dump(total_memory, file)
    with open('saved_weights/reward_' + MODEL_NAME + '.txt', 'wb') as file:
        pickle.dump(total_reward, file)
    print(f"EXITED WITH {episode}th episode")

  0%|          | 0/4995 [00:00<?, ?episodes/s]

2/2 [==============================] - 0s 2ms/step


  0%|          | 0/4995 [00:07<?, ?episodes/s]

EXITED WITH 6th episode
